In [2]:
from aide_design.play import*
import math
from pytexit import py2tex


# Tube Settler Calculations

## Introduction
The tube settler is an integral part of the UASB team's additions to a traditional UASB. The tube settler is modeled after the plate settlers in a sedimentation tank. This design addition will increase the overall solids retention time in the reactor body and improve the effluent quality by allowing particles that leave the body of the reactor time to settle back down. The calculations below determine the design parameters required for the tube settler. 


## Tube Settler Design 

The tube settler will be a larger diameter tube with plate settlers inside it. The tube settler will primariliy extrude from the body of the UASB reactor. 


[Google Drive Image, UASB Reactor Side View](https://docs.google.com/drawings/d/1YEYftDpw_Mzjo6H_TVIifUeApaDGMiU-Yh6jBr0SFlM/edit)

![](UASB_Reactor_Side.jpg)

**Figure 1:** This section will focus on the tube settler used for the effluent shown in the upper right of the diagram of the proposed UASB reactor.

## Design Parameters

Here we set a series of parameters for the design of our pilot scale reactor.  These values are subject to change, but they offer our current ideas on the size and function of our reactor.  The parameters and the reasons for their decision is detailed below.

Parameter| Value | Basis of Design
:------------- |:-------------|:--------
Height of reactor| 7 ft | max allowed to fabricate in lab safely
Diameter of reactor |3 ft | based on 1 L/s plant design
Height of sludge blanket | 3.5 ft| assumed half height of reactor
Distance between plate settlers | 2.5 cm | based on sedimentation tank design 
Angle of tube settler | 60 degrees | based on sedimentation tank design
Thickness of plates | 2 mm| taken from corrugated plastic thickness


## Critical Assumptions

Parameter| Value | Basis of Design
:------------- |:-------------|:--------
Capture velocity|  | set to upflow velocity through the tube settler, or 0.05 mm/s
Length of the tube settler |3 ft | set to the water elevation height in the body of the reactor
Hydraulic residence time | 4 hr | number taken from previous lab scale tests and literature reviews
Diameter sludge weir | 6 inch | variable used for calculations
Sludge weir tube settler separation distance | 12 inch | arbitrary separation distance for ease of fabrication <sup>1</sup> 
Water elevation | 6.5 ft | 6 inches for safety below the full height of the reactor

<sup>1</sup>If this separation distance is too large the tube settler won't be long enough, if it is too low it will touch the sludge blanket.  

In [3]:
def UASBSize(diam, height):
    """Takes the inputs of diameter and height. The bottom of the UASB is sloped
    at 60 degrees with a 3 inch space across the bottom of the UASB. Assumes that half the reactor 
    contains the settled bed, which is used for the HRT. Returns five outputs: 1. height of the sloped
    sides of the bottom geometry, 2. volume of sludge in the reactor, 3. flow rate,
    4. number of people served with graywater, 5. number of people served with blackwater. 
    """
    
    WW_gen = 3 * u.mL/u.s        #Wastewater generated per person, rule of thumb from Monroe
    WW_gen_bw = 0.6 * u.mL/u.s   #Assumes 20% of mixed wastewater
    HRT = 4 * u.hr               #Hydraulic Residence Time, determined from lab scale tests
    
    centerspace = 3 * u.inch     #Center space allows for a 3 inch pipe across the bottom
    phi = math.atan((diam/2)/(centerspace/2))
    angle = 60 * u.deg           #Angle of the sloped bottom
    
    height_cyl_hoof = diam/2 * np.tan(angle)    #Hoof is if 
    height_cyl_wedge = height_cyl_hoof - ((centerspace/2) * math.tan(angle))
    vol_cyl_wedge = height_cyl_wedge * (diam/2)**2 / 3 * ((
        3*math.sin(phi) - 3*phi*math.cos(phi) - math.sin(phi)**3)/(1-math.cos(phi)))
    vol_reactor = (math.pi * (diam / 2)**2 * height / 2) - (2 * vol_cyl_wedge)
    
    flow = vol_reactor / HRT
    people_served = int(flow / WW_gen)       #People served per reactor
    people_served_BW = int(flow / WW_gen_bw) #People served per reactor treating only blackwater
    
    output = [height_cyl_wedge.to(u.m), vol_reactor.to(u.L), flow.to(u.L/u.s), people_served, people_served_BW]
    
    print("The height of the bottom geometry is",height_cyl_wedge.to(u.m))
    print('The volume of the sludge in the reactor is', vol_reactor.to(u.L))
    print('The flow rate of the reactor is', flow.to(u.L/u.s))
    print('The number of people served by this reactor is', people_served)
    print('The number of people served by this reactor if only blackwater is treated is', people_served_BW)
    return output

In [4]:
UASB = UASBSize(3*u.ft, 7*u.ft)

The height of the bottom geometry is 0.7259 meter
The volume of the sludge in the reactor is 520.8 liter
The flow rate of the reactor is 0.03617 liter / second
The number of people served by this reactor is 12
The number of people served by this reactor if only blackwater is treated is 60


In [6]:
# Design Parameters
height_blanket = 3.5 * u.ft 
plate_space = 2.5 * u.cm
angle = 60 * u.deg
thickness_sed_plate = 2 * u.mm
flow = UASB[2]

# Assumptions
diam_sludge_weir = 6 * u.inch
sep_dist = 12 * u.inch
water_elevation = 6.5 * u.ft  ## figure out from previous reports

In [6]:
diam_tube = np.array([8,10]) * u.inch

B = (plate_space + thickness_sed_plate).to(u.cm)

velocity_active_up = (flow * np.sin(angle)/(pc.area_circle(diam_tube))).to(u.mm/u.s)
print("The vertical velocity component beneath the plate settlers is", velocity_active_up.magnitude,velocity_active_up.units )

velocity_plate_up = velocity_active_up * B / plate_space
print("The vertical velocity component between the plate settlers is", velocity_plate_up.magnitude, velocity_plate_up.units)

velocity_plate = (velocity_plate_up / np.sin(angle)).to(u.mm/u.s)
print("The velocity between the plate settlers is", velocity_plate.magnitude, velocity_plate.units)



The vertical velocity component beneath the plate settlers is [ 0.96585625  0.618148  ] millimeter / second
The vertical velocity component between the plate settlers is [ 1.04312475  0.66759984] millimeter / second
The velocity between the plate settlers is [ 1.2044967   0.77087789] millimeter / second


In [7]:

height_tube_settler = (height_blanket + diam_sludge_weir + sep_dist + 0.5*diam_tube).to(u.inch)  # height of the center of the tube setler
print("The height of the center of the tube settler where is attaches to the body of the reactor is",height_tube_settler.magnitude, height_tube_settler.units)

length_tube_settler_vertical = (water_elevation - height_tube_settler).to(u.inch)
print("The vertical length of the tube settler is", length_tube_settler_vertical.magnitude, length_tube_settler_vertical.units)

length_tube_settler = (length_tube_settler_vertical / np.sin(angle)).to(u.cm)
print("The length of the tube setter is",length_tube_settler.magnitude, length_tube_settler.units)

projected_area = (((length_tube_settler * np.cos(angle)
                  ) + (plate_space/np.sin(angle))) * diam_tube).to(u.m**2)
print("The projected area of the plates is", projected_area.magnitude, projected_area.units)

velocity_capture = (plate_space.to(u.mm) * velocity_plate_up.to(u.mm/u.s))/(length_tube_settler.to(u.mm) * np.sin(angle) * np.cos(angle) + plate_space.to(u.mm))
print("The capture veloctiy is", velocity_capture.magnitude, velocity_capture.units)

number_plate_settler = np.floor(diam_tube / (plate_space.to(u.inch) + thickness_sed_plate.to(u.inch)))
print("The number of plate settlers is", number_plate_settler.magnitude, number_plate_settler.units)

The height of the center of the tube settler where is attaches to the body of the reactor is [ 64.  65.] inch
The vertical length of the tube settler is [ 14.  13.] inch
The length of the tube setter is [ 41.06115114  38.12821178] centimeter
The projected area of the plates is [ 0.04758401  0.05575518] meter ** 2
The capture veloctiy is [ 0.12859033  0.08779588] millimeter / second
The number of plate settlers is [ 7.  9.] dimensionless
